In [99]:
from bs4 import BeautifulSoup
import datetime
import requests
import pickle

In [100]:
URL = "https://class-descriptions.northwestern.edu/4880/WCAS/PHIL"
r = requests.get(URL)

In [101]:
soup=BeautifulSoup(r.content)
classes=soup.find('div',attrs={"class":"expander expander1"})


In [121]:
classesLinks=[x.find('a')["href"] for x in classes.find_all('li')]
classesTitles=[" ".join(x.text.strip().replace(" "," ").replace("\n","").split()) for x in classes.find_all('li')]

In [122]:
def getInfo(c):
    c="/".join(URL.split("/")[:-1])+'/'+c
    c = requests.get(c)
    cSoup=BeautifulSoup(c.content)
    for x in cSoup.find_all('p'):
        if(":" in x.text and "-" in x.text and ("am" in x.text or "pm" in x.text)):
            return x.text.replace("\n","").strip()
    return cSoup.find_all('p')[1].text.replace("\n","").strip()

In [123]:
informations=[getInfo(x) for x in classesLinks]

In [124]:
informations

['Kresge 3438 Philosophy Sem. Rm: Tues, Thurs,  11:00AM - 12:20PM',
 'Kresge 3438 Philosophy Sem. Rm: Tues, Thurs,  2:00PM - 3:20PM',
 'In this seminar we will examine some of the fundamental ideas and questions behind democracy and provide a reading of their "inventors". Some of the questions are: What is democracy? Is it a form of government, a value, an ideal, a political system, a form of life, a bit of all this? Is democracy always the best political solution (in wartime? general starvation?)? Why should the whole of the people decide and not the specialists in the respective questions? Are all democratically taken decisions automatically legitimate (what about minorities\\\' rights?)? How should all citizens in a democracy participate in politics? By direct self-government of the people or by voting representatives? Is everything democratically decidable or does the individual have unalterable rights? Is tolerance and/or free speech necessary for democracy and how far can it go?'

In [125]:
locations=[x[:x.index(":"):] for x in informations]

dayKey={"Mon":0,"Tues":1,"Wed":2,"Thurs":3,"Fri":4,"Sat":5,"Sun":6}

days=[x[x.index(":")+2:len(x)-x[::-1].index(",")-1] for x in informations]
days=[cDays.replace(' ','').split(',') for cDays in days]
days=[[dayKey[day] for day in cDays] for cDays in days]

timings=[x[len(x)-x[::-1].index(","):].replace(" ","").split('-') for x in informations]

ValueError: substring not found

In [112]:
def query(today=datetime.datetime.today().weekday(), totime=datetime.datetime.today().time()):
    results=[]
    startTimes=[]
    for ind, i in enumerate(days):
        frm=datetime.datetime.strptime(timings[ind][0],'%I:%M%p').time()
        to=datetime.datetime.strptime(timings[ind][0],'%I:%M%p').time()
        if(today in i and to>=totime):
            startTimes.append(frm)
            results.append([classesTitles[ind],locations[ind]])
    i=[]
    for i in zip( *sorted( zip(startTimes, results) ) ):
        pass
    sortArr = i
    return sortArr

In [119]:
query(4)

[]

In [114]:
with open('classesLinks.pkl', 'rb') as f: classesLinks=pickle.load(f)
with open('classesTitles.pkl', 'rb') as f: classesTitles=pickle.load(f)
with open('locations.pkl', 'rb') as f: locations=pickle.load(f)
with open('days.pkl', 'rb') as f: days=pickle.load(f)
with open('timings.pkl', 'rb') as f: timings=pickle.load(f)

In [116]:
query(0)

(['1: Game Design Studio - Robert Zubek - Mon, Wed, 6:00PM - 7:20PM',
  'Annenberg Hall G15'],
 ['9: Computer Science Education - Special Projects in Computer Science - Michael Stephen Horn - Paula Kay Hooper - Mon, 6:30PM - 9:20PM',
  'Annenberg Hall 303'])